In [1]:
import pyspark.sql.types as typ
import pandas as pd

Cargar dataset

In [2]:
datos = pd.read_csv("allData.csv")

datosDF = sqlContext.createDataFrame(datos)
datosDF=datosDF.drop("Unnamed: 0")


In [3]:
print(datos.loc[[1457]])

      Unnamed: 0 Aire_acondicionado Altura_sótano  Ancho_de_calle  \
1457        1457                  Y            TA            75.0   

      Año_de_construcción  Año_de_reforma    Base  Baños_a_medias_por_encima_  \
1457                 1965            1965  CBlock                           1   

      Baños_a_medias_sótano  Baños_enteros_por_encima      ...       \
1457                    0.0                         1      ...        

      Tipo_de_calefacción Tipo_de_garaje Tipo_de_venta Tipo_de_vivienda  \
1457                 GasA         Attchd            WD             1Fam   

      Total_pies_cuadr_sótano Valla  Valor_otras_caract Vecindario Zona  \
1457                   1256.0  None                   0    Edwards   RL   

     PrecioDeVenta  
1457        147500  

[1 rows x 76 columns]


In [4]:
labels= datosDF.dtypes

In [5]:
def get_dummy(df,categoricalCols,continuousCols,labelCol):
    
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)) for c in categoricalCols ]
    
    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol())) for indexer in indexers ]
    
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders] + continuousCols, outputCol="features")
    
    pipeline = Pipeline(stages=indexers + encoders + [assembler])
    
    model=pipeline.fit(df)
    data = model.transform(df)
    
    data = data.withColumn('label',col(labelCol))

    
    return data.select('features','label')

In [6]:
catcols = catcols = ['Aire_acondicionado','Altura_sótano','Base','Calidad_chimeneas','Calidad_cocina','Calidad_garaje','Calidad_materiales_exterior','Calidad_y_condición_de_la_calefacción','Calidad_y_condición_piscina','Callejón','Clasificación_sótano_zona1','Clasificación_sótano_zona2','Condición_1','Condición_2','Condición_garaje','Condición_materiales_exterior','Condición_sótano','Condición_venta','Estilo_tejado','Funcionalidad_casa','Geografía_del_terreno','Localización','Material_tejado','Otras_características','Pavimento','Pavimento_entrada_coche','Pendiente','Plantas','Primer_revestimiento_exterior','Puertecita_sótano','Relieve','Segundo_revestimiento_exterior','Sistema_eléctrico','Situación_garaje','Tipo_de_bloque','Tipo_de_calefacción','Tipo_de_garaje','Tipo_de_venta','Tipo_de_vivienda','Valla','Vecindario','Zona']


num_cols = ['Ancho_de_calle','Año_de_construcción','Año_de_reforma','Baños_a_medias_por_encima_','Baños_a_medias_sótano','Baños_enteros_por_encima','Baños_enteros_sótano','Calidad_materiales','Calidad_vivienda','Cantidad_bloques_pies_cuadrados','Chimeneas','Clasificación','Coches_garaje','Cocina_por_encima','Dormitorios_por_encima','Pes_cuadr_baja_calidad','Pes_cuadr_totales_(sin_garaje)','Pies_cuadr_cerramiento','Pies_cuadr_cubierta_madera','Pies_cuadr_piscina','Pies_cuadr_poche_abierto','Pies_cuadr_porcha_cerrado','Pies_cuadr_segundo_piso','Pies_cuadr_sótano_sin_acabar','Pies_cuadr_z1_sótano','Pies_cuadr_z2_sótano','Pies_cuadrados_terreno','Precio_mensual','PrecioAnual','ScreenPorch','Total_pies_cuadr_sótano','Valor_otras_caract']
labelCol = 'PrecioDeVenta'

data = get_dummy(datosDF,catcols,num_cols,labelCol)


In [7]:
data.show(5)

+--------------------+------+
|            features| label|
+--------------------+------+
|(253,[0,2,5,10,16...|208500|
|(253,[0,2,6,12,15...|181500|
|(253,[0,2,5,12,16...|223500|
|(253,[0,1,7,11,16...|140000|
|(253,[0,2,5,12,16...|250000|
+--------------------+------+
only showing top 5 rows



In [8]:
from pyspark.ml.feature import StringIndexer
# Index labels, adding metadata to the label column
labelIndexer = StringIndexer(inputCol='label', outputCol='indexedLabel').fit(data)
labelIndexer.transform(data).show(5, True)

+--------------------+------+------------+
|            features| label|indexedLabel|
+--------------------+------+------------+
|(253,[0,2,5,10,16...|208500|       329.0|
|(253,[0,2,6,12,15...|181500|       286.0|
|(253,[0,2,5,12,16...|223500|       212.0|
|(253,[0,1,7,11,16...|140000|         1.0|
|(253,[0,2,5,12,16...|250000|        23.0|
+--------------------+------+------------+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import VectorIndexer
import pyspark.ml.classification as cl

logistic = cl.LogisticRegression(
    maxIter=10,
    regParam=0.01,
    labelCol='indexedLabel')
featureIndexer =VectorIndexer(inputCol="features", \
                              outputCol="indexedFeatures", \
                              maxCategories=4).fit(data)
featureIndexer.transform(data).show(5, True)

+--------------------+------+--------------------+
|            features| label|     indexedFeatures|
+--------------------+------+--------------------+
|(253,[0,2,5,10,16...|208500|(253,[0,2,5,10,16...|
|(253,[0,2,6,12,15...|181500|(253,[0,2,6,12,15...|
|(253,[0,2,5,12,16...|223500|(253,[0,2,5,12,16...|
|(253,[0,1,7,11,16...|140000|(253,[0,1,7,11,16...|
|(253,[0,2,5,12,16...|250000|(253,[0,2,5,12,16...|
+--------------------+------+--------------------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import lit

In [11]:
data = data.withColumn("Id", lit(0))

In [12]:
train = sqlContext.createDataFrame(data.head(1458), data.schema)

In [13]:
test=data.subtract(train)

In [14]:
(trainingData, testData) = train.randomSplit([0.7, 0.3])

In [15]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='indexedFeatures', labelCol='indexedLabel')

In [16]:
logr.getRegParam()


0.0

In [17]:
from pyspark.ml.feature import IndexToString
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="SalePrice",
                              labels=labelIndexer.labels)

In [18]:
from pyspark.ml import Pipeline
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, logr,labelConverter])

In [19]:
# Train model. This also runs the indexers.
model = pipeline.fit(trainingData)

In [20]:
prediction = model.transform(test)

In [21]:
lrModel = model.stages[2]

In [22]:
trainingSummary = lrModel.summary

In [24]:

#trainingSummary4.accuracy

In [25]:
predicted=prediction.select("SalePrice")

In [26]:
predicted.show()


+---------+
|SalePrice|
+---------+
|    87000|
|   130000|
|   210000|
|   147000|
|   125500|
|   185900|
|   143000|
|    67000|
|   128500|
|   137500|
|   110000|
|    55993|
|   446261|
|   301000|
|   133000|
|   159000|
|   154000|
|    80000|
|   167500|
|   139400|
+---------+
only showing top 20 rows



In [27]:
predicted.toPandas().to_csv('mycsv.csv')

In [28]:
import pyspark.ml.tuning as tune
grid = tune.ParamGridBuilder().addGrid(logistic.maxIter, [100,150,200]).addGrid(logr.regParam, [0.01, 0.05, 0.3]).build()

In [29]:
import pyspark.ml.evaluation as ev

In [30]:
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='label')

In [31]:
cv = tune.CrossValidator(estimator=logistic, estimatorParamMaps=grid, evaluator=evaluator)

In [32]:
data_transformer = model.transform(train)

In [33]:
data_transformer = data_transformer.drop('prediction')
data_transformer = data_transformer.drop('rawPrediction')
data_transformer = data_transformer.drop('probability')

In [35]:
#cvModel = cv.fit(data_transformer)

In [36]:
#print ('Best Param (MaxIter): ', cvModel.bestModel._java_obj.getMaxIter())

In [37]:
#print('Best Param (Max): ', cvModel.bestModel._java_obj.getRegParam())

In [38]:
logr2 = LogisticRegression(featuresCol='indexedFeatures', labelCol='indexedLabel',maxIter=50,regParam=0.01)

In [39]:
pipeline4 = Pipeline(stages=[labelIndexer, featureIndexer, logr2,labelConverter])

In [40]:
model4 = pipeline4.fit(trainingData)

In [42]:
#trainingSummary2 = lrModel2.summary

In [43]:
prediction4 = model4.transform(test)

In [44]:
predicted4=prediction.select("SalePrice")

In [45]:
predicted4.show()

+---------+
|SalePrice|
+---------+
|    87000|
|   130000|
|   210000|
|   147000|
|   125500|
|   185900|
|   143000|
|    67000|
|   128500|
|   137500|
|   110000|
|    55993|
|   446261|
|   301000|
|   133000|
|   159000|
|   154000|
|    80000|
|   167500|
|   139400|
+---------+
only showing top 20 rows



In [46]:
predicted4.toPandas().to_csv('mycsv4.csv')

In [47]:
from pyspark.ml.classification import RandomForestClassifier


In [48]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=5)

In [51]:

pipeline3 = Pipeline(stages=[labelIndexer, featureIndexer, rf,labelConverter])

In [52]:

model3 = pipeline3.fit(trainingData)

In [53]:
predictions3 = model3.transform(test)

In [54]:
predicted3=predictions3.select("Id","SalePrice")

In [55]:
predicted3.show()

+---+---------+
| Id|SalePrice|
+---+---------+
|  0|   125000|
|  0|   155000|
|  0|   140000|
|  0|   230000|
|  0|   124000|
|  0|   230000|
|  0|   208900|
|  0|    87000|
|  0|   135000|
|  0|   140000|
|  0|   135000|
|  0|   125000|
|  0|   230000|
|  0|   155000|
|  0|   165500|
|  0|   144000|
|  0|   155000|
|  0|   135000|
|  0|   250000|
|  0|   135000|
+---+---------+
only showing top 20 rows



In [56]:
predicted3.toPandas().to_csv('mycsv3.csv')

Arbol

In [57]:
from pyspark.ml.classification import DecisionTreeClassifier

dTree = DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='indexedFeatures')

In [58]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="SalePrice",
                               labels=labelIndexer.labels)
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dTree,labelConverter])
# Train model.  This also runs the indexers.
model2 = pipeline.fit(trainingData)

In [59]:
prediction2 = model2.transform(test)


In [60]:
predicted2=prediction2.select("Id","SalePrice")

In [61]:
predicted2.toPandas().to_csv('mycsv2.csv')

In [62]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

In [70]:
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                             outputCol="selectedFeatures", labelCol="label")



In [71]:
result = selector.fit(train)

print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())


ChiSqSelector output with top 10 features selected


In [72]:
importantFeatures = result.selectedFeatures

In [73]:
importantFeatures

[3, 17, 23, 24, 25, 61, 76, 79, 178, 198]

In [74]:
train.select('features').show()

+--------------------+
|            features|
+--------------------+
|(253,[0,2,5,10,16...|
|(253,[0,2,6,12,15...|
|(253,[0,2,5,12,16...|
|(253,[0,1,7,11,16...|
|(253,[0,2,5,12,16...|
|(253,[0,2,10,15,1...|
|(253,[0,3,5,11,16...|
|(253,[0,2,6,12,15...|
|(253,[0,1,7,12,15...|
|(253,[0,1,7,12,15...|
|(253,[0,1,6,10,15...|
|(253,[0,3,5,11,17...|
|(253,[0,1,6,10,15...|
|(253,[0,2,5,11,16...|
|(253,[0,1,6,13,15...|
|(253,[0,1,7,10,15...|
|(253,[0,1,6,12,15...|
|(253,[0,8,10,15,1...|
|(253,[0,1,5,10,16...|
|(253,[0,1,6,10,15...|
+--------------------+
only showing top 20 rows



In [75]:
from pyspark.ml.feature import PCA

In [76]:
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
model = pca.fit(train)
model.transform(train).collect()[0].pca_features


DenseVector([-8484.7605, 1902.1165])

In [77]:
model.read

<bound method JavaMLReadable.read of <class 'pyspark.ml.feature.PCAModel'>>